### Multimodal(图片) 分析

场景：出租车内摄像头观测安全带

<table align="left">
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2Fcloudymoma%2Fgcp-playgroud-public%2Frefs%2Fheads%2Fmaster%2FBigQuery%2Ftaxi%2Ftaxi_img_seatbelt.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Run in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/cloudymoma/gcp-playgroud-public/blob/master/BigQuery/taxi/taxi_img_seatbelt.ipynb">
      <img width="32px" src="https://upload.wikimedia.org/wikipedia/commons/9/91/Octicons-mark-github.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

In [ ]:
%%bigquery

CREATE OR REPLACE EXTERNAL TABLE `du-hast-mich.taxi.camera_shots`
WITH CONNECTION `projects/du-hast-mich/locations/us/connections/bq_img_con`
OPTIONS (
  object_metadata = 'SIMPLE',
  uris = ['gs://dingotaxi/img_seatbelt/*']
);

In [ ]:
%%bigquery

CREATE OR REPLACE MODEL `du-hast-mich.taxi.gemini`
REMOTE WITH CONNECTION `us.bq_img_con`
OPTIONS (endpoint = 'gemini-2.5-flash');


In [ ]:
%%bigquery

CREATE OR REPLACE TABLE `du-hast-mich.taxi.10_camera_shots_seatbelt` AS
SELECT
  * EXCEPT(ml_generate_text_result, ml_generate_text_status),
  -- 使用 CASE WHEN 逻辑来生成 String 类型的标签
  CASE
    -- 将结果转为小写并去空格，匹配 true
    WHEN LOWER(TRIM(JSON_VALUE(ml_generate_text_result, '$.candidates[0].content.parts[0].text'))) = 'true' THEN 'True'
    -- 匹配 false
    WHEN LOWER(TRIM(JSON_VALUE(ml_generate_text_result, '$.candidates[0].content.parts[0].text'))) = 'false' THEN 'False'
    -- 兜底逻辑：任何不是 true/false 的输出（包括模型说 "unsure" 或其他废话）都标记为 suspicious
    ELSE 'suspicious'
  END AS seatbelt
FROM
  ML.GENERATE_TEXT(
    MODEL `du-hast-mich.taxi.gemini`,
    TABLE `du-hast-mich.taxi.camera_shots`,
    STRUCT(
      -- 优化 Prompt：明确允许模型在不确定时输出 "suspicious"
      'Observe the driver/passenger in the image. Are they wearing a seatbelt correctly? Answer only "true" if yes, "false" if no. If you are unsure or the image is unclear, answer "suspicious". Do not use Markdown.' AS prompt,
      0.2 AS temperature,
      10 AS max_output_tokens
    )
  );

In [ ]:
from google.cloud import bigquery
import matplotlib.pyplot as plt

# 初始化 BigQuery 客户端
client = bigquery.Client()

# 1. 构建查询语句：统计 True, False, suspicious 的数量
query = """
    SELECT seatbelt, COUNT(*) as count
    FROM `du-hast-mich.taxi.10_camera_shots_seatbelt`
    GROUP BY seatbelt
"""

# 2. 执行查询并将结果转换为 Pandas DataFrame
df = client.query(query).to_dataframe()

# 3. 绘制饼图 (Pie Chart)
plt.figure(figsize=(8, 8))

# 定义颜色映射，使图表更直观（绿色=True, 红色=False, 灰色=suspicious）
colors_map = {'True': '#66b3ff', 'False': '#ff9999', 'suspicious': '#999999'}
# 根据数据中的实际标签顺序匹配颜色，如果没有对应标签则使用默认色
colors = [colors_map.get(x, '#ffcc99') for x in df['seatbelt']]

plt.pie(
    df['count'],
    labels=df['seatbelt'],      # 显示类别标签 (True/False/suspicious)
    autopct='%1.1f%%',          # 只显示百分比，保留一位小数
    startangle=140,
    colors=colors,
    textprops={'fontsize': 14}  # 调整字体大小
)

plt.title('Seatbelt Usage Detection Results')
plt.axis('equal')  # 保证饼图是正圆形
plt.show()